In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Academic publication style with larger fonts
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif', 'serif'],
    'font.size': 13,
    'axes.titlesize': 15,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 16,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'grid.alpha': 0.3,
    'axes.linewidth': 1.2,
    'xtick.major.width': 1.2,
    'ytick.major.width': 1.2,
})

# Paths
PROJECT_ROOT = Path('..').resolve()
RESULTS_DIR = PROJECT_ROOT / 'results' / 'metrics' / 'experiment2_with_artist'
FIGURES_DIR = PROJECT_ROOT / 'results' / 'figures'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Figures will be saved to: {FIGURES_DIR}")

## 1. Load Test Evaluation Results

In [ ]:
# Find the most recent test evaluation results
test_results_files = sorted(RESULTS_DIR.glob('test_evaluation_final_*.csv'))

if not test_results_files:
    raise FileNotFoundError("No test evaluation results found. Run test_evaluation_final.py first.")

# Load most recent results
results_file = test_results_files[-1]
df_test = pd.read_csv(results_file)

print(f"Loaded: {results_file.name}")
print(f"Shape: {df_test.shape}")
print(f"\nColumns: {df_test.columns.tolist()}")
print(f"\nFirst few rows:")
df_test.head(10)

In [ ]:
# Basic statistics
print("="*80)
print("TEST EVALUATION OVERVIEW")
print("="*80)

print(f"\nTotal evaluations: {len(df_test)}")
print(f"Targets evaluated: {df_test['target'].unique().tolist()}")
print(f"Models evaluated: {sorted(df_test['model'].unique().tolist())}")
print(f"Model sources: {df_test['model_source'].unique().tolist()}")
print(f"\nTest samples: {df_test['n_samples'].iloc[0]:,}")

print("\nEvaluations by model source:")
print(df_test['model_source'].value_counts())

print("\nEvaluations by target:")
print(df_test['target'].value_counts())

## 2. Summary Statistics

In [ ]:
# Overall statistics
print("="*80)
print("OVERALL PERFORMANCE STATISTICS")
print("="*80)

summary_stats = df_test.groupby('model_source')[['r2', 'rmse', 'mae']].agg(['mean', 'std', 'min', 'max'])
print("\nAcross all models and targets:")
print(summary_stats.round(4))

print("\n" + "="*80)
print("PERFORMANCE BY TARGET")
print("="*80)

target_summary = df_test.groupby(['target', 'model_source'])[['r2', 'rmse', 'mae']].mean().round(4)
print(target_summary)

## 3. Best Models per Target

In [ ]:
# Best model for each target (overall)
print("="*80)
print("BEST MODELS PER TARGET (HIGHEST R²)")
print("="*80)

best_models_overall = df_test.loc[df_test.groupby('target')['r2'].idxmax()]
best_display = best_models_overall[[
    'target', 'model', 'model_source', 'num_features', 'r2', 'rmse', 'mae'
]].sort_values('r2', ascending=False)

print(best_display.to_string(index=False))

# Best enhanced vs best RFE per target
print("\n" + "="*80)
print("BEST ENHANCED VS BEST RFE PER TARGET")
print("="*80)

for target in sorted(df_test['target'].unique()):
    target_df = df_test[df_test['target'] == target]
    
    # Best enhanced
    enhanced_df = target_df[target_df['model_source'] == 'enhanced']
    if len(enhanced_df) > 0:
        best_enhanced = enhanced_df.loc[enhanced_df['r2'].idxmax()]
    
    # Best RFE
    rfe_df = target_df[target_df['model_source'] == 'rfe']
    if len(rfe_df) > 0:
        best_rfe = rfe_df.loc[rfe_df['r2'].idxmax()]
    
    print(f"\n{target.upper()}:")
    if len(enhanced_df) > 0:
        print(f"  Enhanced ({int(best_enhanced['num_features'])} features):")
        print(f"    {best_enhanced['model']:20s} R²={best_enhanced['r2']:.4f}, RMSE={best_enhanced['rmse']:.4f}")
    
    if len(rfe_df) > 0:
        print(f"  RFE ({int(best_rfe['num_features'])} features):")
        print(f"    {best_rfe['model']:20s} R²={best_rfe['r2']:.4f}, RMSE={best_rfe['rmse']:.4f}")
    
    if len(enhanced_df) > 0 and len(rfe_df) > 0:
        r2_diff = best_rfe['r2'] - best_enhanced['r2']
        feature_reduction = (1 - best_rfe['num_features'] / best_enhanced['num_features']) * 100
        print(f"  Comparison:")
        print(f"    ΔR² = {r2_diff:+.4f} ({(r2_diff/best_enhanced['r2']*100):+.2f}%)")
        print(f"    Feature reduction: {feature_reduction:.1f}%")

## 4. Enhanced vs RFE Comparison Visualizations

In [ ]:
# R² comparison - Enhanced vs RFE (grouped by target)
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

targets = sorted(df_test['target'].unique())
colors_enhanced = '#3498db'
colors_rfe = '#e74c3c'

for idx, target in enumerate(targets):
    target_df = df_test[df_test['target'] == target]
    
    # Get unique models
    models = sorted(target_df['model'].unique())
    
    # Prepare data
    enhanced_r2 = []
    rfe_r2 = []
    
    for model in models:
        model_df = target_df[target_df['model'] == model]
        
        enhanced_val = model_df[model_df['model_source'] == 'enhanced']['r2'].values
        enhanced_r2.append(enhanced_val[0] if len(enhanced_val) > 0 else 0)
        
        rfe_val = model_df[model_df['model_source'] == 'rfe']['r2'].values
        rfe_r2.append(rfe_val[0] if len(rfe_val) > 0 else 0)
    
    # Plot grouped bars
    x = np.arange(len(models))
    width = 0.38
    
    bars1 = axes[idx].bar(x - width/2, enhanced_r2, width, label='Enhanced (414 features)',
                         color=colors_enhanced, alpha=0.85, edgecolor='black', linewidth=1)
    bars2 = axes[idx].bar(x + width/2, rfe_r2, width, label='RFE (reduced features)',
                         color=colors_rfe, alpha=0.85, edgecolor='black', linewidth=1)
    
    axes[idx].set_ylabel('R² Score', fontweight='bold')
    axes[idx].set_title(f'{target.capitalize()}', fontweight='bold', fontsize=16, pad=10)
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(models, rotation=45, ha='right', fontsize=10)
    axes[idx].legend(loc='lower right', frameon=True, fancybox=False, 
                    edgecolor='black', framealpha=0.95)
    axes[idx].grid(True, alpha=0.25, linestyle='--', axis='y')
    axes[idx].set_ylim([0, max(max(enhanced_r2), max(rfe_r2)) * 1.15])
    
    # Add value labels on bars
    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        if enhanced_r2[i] > 0:
            axes[idx].text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 0.01,
                          f'{enhanced_r2[i]:.3f}', ha='center', va='bottom', fontsize=8)
        if rfe_r2[i] > 0:
            axes[idx].text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 0.01,
                          f'{rfe_r2[i]:.3f}', ha='center', va='bottom', fontsize=8)

plt.suptitle('Test Set Performance: Enhanced vs RFE Models (R² Comparison)',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_r2_enhanced_vs_rfe_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_r2_enhanced_vs_rfe_comparison.png")

In [ ]:
# R² visualization - enhanced models
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

for idx, target in enumerate(targets):
    target_df = df_test[df_test['target'] == target]
    enhanced_df = target_df[target_df['model_source'] == 'enhanced']
    
    if len(enhanced_df) > 0:
        # Sort by R² score
        enhanced_sorted = enhanced_df.sort_values('r2', ascending=True)
        models = enhanced_sorted['model'].values
        r2_values = enhanced_sorted['r2'].values
        
        # Color by tuning status
        colors = ['#27ae60' if '_tuned' in m else '#e74c3c' for m in models]
        
        bars = axes[idx].barh(range(len(models)), r2_values, color=colors,
                             alpha=0.85, edgecolor='black', linewidth=1.2)
        
        axes[idx].set_yticks(range(len(models)))
        axes[idx].set_yticklabels(models, fontsize=13)
        axes[idx].set_xlabel('R² Score', fontweight='bold')
        axes[idx].set_title(f'{target.capitalize()}',
                           fontweight='bold', fontsize=16, pad=10)
        axes[idx].grid(True, alpha=0.25, linestyle='--', axis='x')
        axes[idx].set_xlim([0, max(r2_values) * 1.15])
        
        # Add value labels
        for i, (bar, val) in enumerate(zip(bars, r2_values)):
            axes[idx].text(val + 0.008, i, f'{val:.4f}', va='center',
                          fontsize=12, fontweight='bold')

# Add legend
legend_elements = [
    Patch(facecolor='#e74c3c', edgecolor='black', label='Default', alpha=0.85),
    Patch(facecolor='#27ae60', edgecolor='black', label='Tuned', alpha=0.85)
]
if len(enhanced_df) > 0:
    axes[0].legend(handles=legend_elements, loc='upper left', frameon=True,
                  fancybox=False, edgecolor='black', framealpha=0.95)

plt.suptitle('Test Set Performance',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_r2_enhanced_models.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_r2_enhanced_models.png")

In [ ]:
# RMSE visualization - enhanced models
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

for idx, target in enumerate(targets):
    target_df = df_test[df_test['target'] == target]
    enhanced_df = target_df[target_df['model_source'] == 'enhanced']
    
    if len(enhanced_df) > 0:
        # Sort by RMSE (ascending = best first)
        enhanced_sorted = enhanced_df.sort_values('rmse', ascending=False)
        models = enhanced_sorted['model'].values
        rmse_values = enhanced_sorted['rmse'].values
        
        # Color by tuning status
        colors = ['#27ae60' if '_tuned' in m else '#e74c3c' for m in models]
        
        bars = axes[idx].barh(range(len(models)), rmse_values, color=colors,
                             alpha=0.85, edgecolor='black', linewidth=1.2)
        
        axes[idx].set_yticks(range(len(models)))
        axes[idx].set_yticklabels(models, fontsize=13)
        axes[idx].set_xlabel('RMSE', fontweight='bold')
        axes[idx].set_title(f'{target.capitalize()}',
                           fontweight='bold', fontsize=16, pad=10)
        axes[idx].grid(True, alpha=0.25, linestyle='--', axis='x')
        
        # Add value labels
        for i, (bar, val) in enumerate(zip(bars, rmse_values)):
            axes[idx].text(val + 0.003, i, f'{val:.4f}', va='center',
                          fontsize=12, fontweight='bold')

# Add legend
legend_elements = [
    Patch(facecolor='#e74c3c', edgecolor='black', label='Default', alpha=0.85),
    Patch(facecolor='#27ae60', edgecolor='black', label='Tuned', alpha=0.85)
]
if len(enhanced_df) > 0:
    axes[0].legend(handles=legend_elements, loc='upper left', frameon=True,
                  fancybox=False, edgecolor='black', framealpha=0.95)

plt.suptitle('Test Set RMSE Performance',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_rmse_enhanced_models.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_rmse_enhanced_models.png")

In [ ]:
# R² visualization - RFE models
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

for idx, target in enumerate(targets):
    target_df = df_test[df_test['target'] == target]
    rfe_df = target_df[target_df['model_source'] == 'rfe']
    
    if len(rfe_df) > 0:
        # Sort by R² score
        rfe_sorted = rfe_df.sort_values('r2', ascending=True)
        models = rfe_sorted['model'].values
        r2_values = rfe_sorted['r2'].values
        num_features = rfe_sorted['num_features'].values
        
        # Color by tuning status
        colors = ['#27ae60' if '_tuned' in m else '#e74c3c' for m in models]
        
        bars = axes[idx].barh(range(len(models)), r2_values, color=colors,
                             alpha=0.85, edgecolor='black', linewidth=1.2)
        
        axes[idx].set_yticks(range(len(models)))
        axes[idx].set_yticklabels(models, fontsize=13)
        axes[idx].set_xlabel('R² Score', fontweight='bold')
        axes[idx].set_title(f'{target.capitalize()} ({int(num_features[0])} features)',
                           fontweight='bold', fontsize=16, pad=10)
        axes[idx].grid(True, alpha=0.25, linestyle='--', axis='x')
        axes[idx].set_xlim([0, max(r2_values) * 1.15])
        
        # Add value labels
        for i, (bar, val) in enumerate(zip(bars, r2_values)):
            axes[idx].text(val + 0.008, i, f'{val:.4f}', va='center',
                          fontsize=12, fontweight='bold')

# Add legend
legend_elements = [
    Patch(facecolor='#e74c3c', edgecolor='black', label='Default', alpha=0.85),
    Patch(facecolor='#27ae60', edgecolor='black', label='Tuned', alpha=0.85)
]
if len(rfe_df) > 0:
    axes[0].legend(handles=legend_elements, loc='upper left', frameon=True,
                  fancybox=False, edgecolor='black', framealpha=0.95)

plt.suptitle('Test Set Performance: Feature Selected',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_r2_rfe_models.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_r2_rfe_models.png")

In [ ]:
# RMSE visualization - RFE models
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

for idx, target in enumerate(targets):
    target_df = df_test[df_test['target'] == target]
    rfe_df = target_df[target_df['model_source'] == 'rfe']
    
    if len(rfe_df) > 0:
        # Sort by RMSE (ascending = best first)
        rfe_sorted = rfe_df.sort_values('rmse', ascending=False)
        models = rfe_sorted['model'].values
        rmse_values = rfe_sorted['rmse'].values
        num_features = rfe_sorted['num_features'].values
        
        # Color by tuning status
        colors = ['#27ae60' if '_tuned' in m else '#e74c3c' for m in models]
        
        bars = axes[idx].barh(range(len(models)), rmse_values, color=colors,
                             alpha=0.85, edgecolor='black', linewidth=1.2)
        
        axes[idx].set_yticks(range(len(models)))
        axes[idx].set_yticklabels(models, fontsize=13)
        axes[idx].set_xlabel('RMSE', fontweight='bold')
        axes[idx].set_title(f'{target.capitalize()} ({int(num_features[0])} features)',
                           fontweight='bold', fontsize=16, pad=10)
        axes[idx].grid(True, alpha=0.25, linestyle='--', axis='x')
        
        # Add value labels
        for i, (bar, val) in enumerate(zip(bars, rmse_values)):
            axes[idx].text(val + 0.003, i, f'{val:.4f}', va='center',
                          fontsize=12, fontweight='bold')

# Add legend
legend_elements = [
    Patch(facecolor='#e74c3c', edgecolor='black', label='Default', alpha=0.85),
    Patch(facecolor='#27ae60', edgecolor='black', label='Tuned', alpha=0.85)
]
if len(rfe_df) > 0:
    axes[0].legend(handles=legend_elements, loc='upper left', frameon=True,
                  fancybox=False, edgecolor='black', framealpha=0.95)

plt.suptitle('Test Set RMSE Performance: Feature Selection',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_rmse_rfe_models.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_rmse_rfe_models.png")

In [ ]:
# RMSE comparison - Enhanced vs RFE
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

for idx, target in enumerate(targets):
    target_df = df_test[df_test['target'] == target]
    models = sorted(target_df['model'].unique())
    
    # Prepare data
    enhanced_rmse = []
    rfe_rmse = []
    
    for model in models:
        model_df = target_df[target_df['model'] == model]
        
        enhanced_val = model_df[model_df['model_source'] == 'enhanced']['rmse'].values
        enhanced_rmse.append(enhanced_val[0] if len(enhanced_val) > 0 else 0)
        
        rfe_val = model_df[model_df['model_source'] == 'rfe']['rmse'].values
        rfe_rmse.append(rfe_val[0] if len(rfe_val) > 0 else 0)
    
    # Plot grouped bars
    x = np.arange(len(models))
    width = 0.38
    
    bars1 = axes[idx].bar(x - width/2, enhanced_rmse, width, label='Enhanced (414 features)',
                         color=colors_enhanced, alpha=0.85, edgecolor='black', linewidth=1)
    bars2 = axes[idx].bar(x + width/2, rfe_rmse, width, label='RFE (reduced features)',
                         color=colors_rfe, alpha=0.85, edgecolor='black', linewidth=1)
    
    axes[idx].set_ylabel('RMSE (Lower is Better)', fontweight='bold')
    axes[idx].set_title(f'{target.capitalize()}', fontweight='bold', fontsize=16, pad=10)
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(models, rotation=45, ha='right', fontsize=10)
    axes[idx].legend(loc='upper right', frameon=True, fancybox=False,
                    edgecolor='black', framealpha=0.95)
    axes[idx].grid(True, alpha=0.25, linestyle='--', axis='y')
    
    # Add value labels
    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        if enhanced_rmse[i] > 0:
            axes[idx].text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 0.003,
                          f'{enhanced_rmse[i]:.3f}', ha='center', va='bottom', fontsize=8)
        if rfe_rmse[i] > 0:
            axes[idx].text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 0.003,
                          f'{rfe_rmse[i]:.3f}', ha='center', va='bottom', fontsize=8)

plt.suptitle('Test Set Performance: Enhanced vs RFE Models (RMSE Comparison)',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_rmse_enhanced_vs_rfe_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_rmse_enhanced_vs_rfe_comparison.png")

In [ ]:
# RFE models heatmap
rfe_df = df_test[df_test['model_source'] == 'rfe']
if len(rfe_df) > 0:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    pivot_rfe = rfe_df.pivot(index='model', columns='target', values='r2')
    pivot_rfe = pivot_rfe[targets]
    pivot_rfe = pivot_rfe.sort_values(pivot_rfe.columns.tolist(), ascending=False)
    
    sns.heatmap(pivot_rfe, annot=True, fmt='.4f', cmap='RdYlGn', center=0.35,
                ax=ax, linewidths=1.5, vmin=0, vmax=0.9, cbar_kws={'label': 'R² Score'},
                annot_kws={'size': 12, 'weight': 'bold'})
    ax.set_title('RFE Models (34-394 features per target)', fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Target Variable', fontsize=15, fontweight='bold')
    ax.set_ylabel('Model', fontsize=15, fontweight='bold')
    ax.tick_params(axis='x', labelsize=13)
    ax.tick_params(axis='y', labelsize=12)
    
    plt.tight_layout()
    fig.savefig(FIGURES_DIR / 'test_heatmap_rfe.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Saved: test_heatmap_rfe.png")

## 5. Performance Heatmaps

In [ ]:
# Create side-by-side heatmaps for enhanced and RFE
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Enhanced models heatmap
enhanced_df = df_test[df_test['model_source'] == 'enhanced']
if len(enhanced_df) > 0:
    pivot_enhanced = enhanced_df.pivot(index='model', columns='target', values='r2')
    pivot_enhanced = pivot_enhanced[targets]  # Ensure consistent order
    pivot_enhanced = pivot_enhanced.sort_values(pivot_enhanced.columns.tolist(), ascending=False)
    
    sns.heatmap(pivot_enhanced, annot=True, fmt='.4f', cmap='RdYlGn', center=0.35,
                ax=axes[0], linewidths=1, vmin=0, vmax=0.9, cbar_kws={'label': 'R² Score'},
                annot_kws={'size': 11, 'weight': 'bold'})
    axes[0].set_title('Enhanced Models (414 features)', fontsize=16, fontweight='bold', pad=15)
    axes[0].set_xlabel('Target Variable', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Model', fontsize=14, fontweight='bold')
    axes[0].tick_params(axis='x', labelsize=12)
    axes[0].tick_params(axis='y', labelsize=11)

# RFE models heatmap
rfe_df = df_test[df_test['model_source'] == 'rfe']
if len(rfe_df) > 0:
    pivot_rfe = rfe_df.pivot(index='model', columns='target', values='r2')
    pivot_rfe = pivot_rfe[targets]
    pivot_rfe = pivot_rfe.sort_values(pivot_rfe.columns.tolist(), ascending=False)
    
    sns.heatmap(pivot_rfe, annot=True, fmt='.4f', cmap='RdYlGn', center=0.35,
                ax=axes[1], linewidths=1, vmin=0, vmax=0.9, cbar_kws={'label': 'R² Score'},
                annot_kws={'size': 11, 'weight': 'bold'})
    axes[1].set_title('RFE Models (34-394 features)', fontsize=16, fontweight='bold', pad=15)
    axes[1].set_xlabel('Target Variable', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Model', fontsize=14, fontweight='bold')
    axes[1].tick_params(axis='x', labelsize=12)
    axes[1].tick_params(axis='y', labelsize=11)

plt.suptitle('Test Set R² Scores - Enhanced vs RFE', fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_heatmap_enhanced_vs_rfe.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_heatmap_enhanced_vs_rfe.png")

## 6. Feature Efficiency Analysis

In [ ]:
# Analyze R² per feature (efficiency metric)
print("="*80)
print("FEATURE EFFICIENCY ANALYSIS")
print("="*80)
print("\nR² per feature (higher = more efficient):")
print()

efficiency_data = []

for target in targets:
    target_df = df_test[df_test['target'] == target]
    
    # Best enhanced
    enhanced_df = target_df[target_df['model_source'] == 'enhanced']
    if len(enhanced_df) > 0:
        best_enhanced = enhanced_df.loc[enhanced_df['r2'].idxmax()]
        enhanced_efficiency = best_enhanced['r2'] / best_enhanced['num_features']
    
    # Best RFE
    rfe_df = target_df[target_df['model_source'] == 'rfe']
    if len(rfe_df) > 0:
        best_rfe = rfe_df.loc[rfe_df['r2'].idxmax()]
        rfe_efficiency = best_rfe['r2'] / best_rfe['num_features']
    
    if len(enhanced_df) > 0 and len(rfe_df) > 0:
        efficiency_data.append({
            'target': target,
            'enhanced_r2': best_enhanced['r2'],
            'enhanced_features': int(best_enhanced['num_features']),
            'enhanced_efficiency': enhanced_efficiency,
            'rfe_r2': best_rfe['r2'],
            'rfe_features': int(best_rfe['num_features']),
            'rfe_efficiency': rfe_efficiency,
            'efficiency_gain': (rfe_efficiency / enhanced_efficiency - 1) * 100
        })
        
        print(f"{target.upper()}:")
        print(f"  Enhanced: R²={best_enhanced['r2']:.4f} / {int(best_enhanced['num_features'])} features = {enhanced_efficiency:.6f}")
        print(f"  RFE:      R²={best_rfe['r2']:.4f} / {int(best_rfe['num_features'])} features = {rfe_efficiency:.6f}")
        print(f"  Efficiency gain: {(rfe_efficiency / enhanced_efficiency - 1) * 100:+.1f}%")
        print()

efficiency_df = pd.DataFrame(efficiency_data)

In [ ]:
# Visualize RFE models feature efficiency
if len(efficiency_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    x = np.arange(len(efficiency_df))
    
    # Plot 1: R² with feature count
    bars = axes[0].bar(x, efficiency_df['rfe_r2'], color='#e74c3c',
                      alpha=0.85, edgecolor='black', linewidth=1.5)
    
    axes[0].set_ylabel('R² Score', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('Target', fontweight='bold', fontsize=14)
    axes[0].set_title('RFE Models Performance', fontweight='bold', fontsize=16, pad=15)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels([t.capitalize() for t in efficiency_df['target']], fontsize=13)
    axes[0].grid(True, alpha=0.25, linestyle='--', axis='y')
    
    # Add value and feature count labels
    for i, (bar, val, feats) in enumerate(zip(bars, efficiency_df['rfe_r2'], 
                                               efficiency_df['rfe_features'])):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f"{val:.4f}\n({int(feats)} feat)", ha='center', va='bottom',
                    fontsize=12, fontweight='bold')
    
    # Plot 2: Feature efficiency (R² per feature)
    bars = axes[1].bar(x, efficiency_df['rfe_efficiency'] * 1000, color='#e74c3c',
                      alpha=0.85, edgecolor='black', linewidth=1.5)
    
    axes[1].set_ylabel('R² per Feature (×1000)', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('Target', fontweight='bold', fontsize=14)
    axes[1].set_title('RFE Feature Efficiency', fontweight='bold', fontsize=16, pad=15)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels([t.capitalize() for t in efficiency_df['target']], fontsize=13)
    axes[1].grid(True, alpha=0.25, linestyle='--', axis='y')
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, efficiency_df['rfe_efficiency'] * 1000)):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                    f"{val:.2f}", ha='center', va='bottom',
                    fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    fig.savefig(FIGURES_DIR / 'test_rfe_efficiency.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Saved: test_rfe_efficiency.png")

In [ ]:
# RFE models ranking
fig, ax = plt.subplots(figsize=(10, 6))

rfe_avg = df_test[df_test['model_source'] == 'rfe'].groupby('model')['r2'].mean().sort_values(ascending=False)
colors_bar = ['#27ae60' if '_tuned' in m else '#e74c3c' for m in rfe_avg.index]

bars = ax.barh(range(len(rfe_avg)), rfe_avg.values, color=colors_bar,
               alpha=0.85, edgecolor='black', linewidth=1.5)
ax.set_yticks(range(len(rfe_avg)))
ax.set_yticklabels(rfe_avg.index, fontsize=11, fontweight='bold')
ax.set_xlabel('Average R² (across all targets)', fontweight='bold', fontsize=14)
ax.set_title('RFE Models Performance Ranking\n(34-394 features per target)', 
            fontweight='bold', fontsize=16, pad=15)
ax.grid(True, alpha=0.25, linestyle='--', axis='x')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, rfe_avg.values)):
    ax.text(val + 0.005, i, f'{val:.4f}', va='center', fontsize=10, fontweight='bold')

# Legend
legend_elements = [
    Patch(facecolor='#e74c3c', edgecolor='black', label='Default', alpha=0.85),
    Patch(facecolor='#27ae60', edgecolor='black', label='Tuned', alpha=0.85)
]
ax.legend(handles=legend_elements, loc='upper left', frameon=True,
         fancybox=False, edgecolor='black', framealpha=0.95, fontsize=12)

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_rfe_ranking.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_rfe_ranking.png")

In [ ]:
# Visualize feature efficiency
if len(efficiency_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Number of features
    x = np.arange(len(efficiency_df))
    width = 0.35
    
    bars1 = axes[0].bar(x - width/2, efficiency_df['enhanced_features'], width,
                       label='Enhanced', color=colors_enhanced, alpha=0.85,
                       edgecolor='black', linewidth=1.2)
    bars2 = axes[0].bar(x + width/2, efficiency_df['rfe_features'], width,
                       label='RFE', color=colors_rfe, alpha=0.85,
                       edgecolor='black', linewidth=1.2)
    
    axes[0].set_ylabel('Number of Features', fontweight='bold')
    axes[0].set_xlabel('Target', fontweight='bold')
    axes[0].set_title('Feature Count: Enhanced vs RFE', fontweight='bold', fontsize=15)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels([t.capitalize() for t in efficiency_df['target']])
    axes[0].legend(frameon=True, fancybox=False, edgecolor='black', framealpha=0.95)
    axes[0].grid(True, alpha=0.25, linestyle='--', axis='y')
    
    # Add value labels
    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        axes[0].text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 5,
                    f"{int(bar1.get_height())}", ha='center', va='bottom',
                    fontsize=11, fontweight='bold')
        axes[0].text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 5,
                    f"{int(bar2.get_height())}", ha='center', va='bottom',
                    fontsize=11, fontweight='bold')
    
    # Plot 2: Efficiency (R² per feature)
    bars1 = axes[1].bar(x - width/2, efficiency_df['enhanced_efficiency'] * 1000, width,
                       label='Enhanced', color=colors_enhanced, alpha=0.85,
                       edgecolor='black', linewidth=1.2)
    bars2 = axes[1].bar(x + width/2, efficiency_df['rfe_efficiency'] * 1000, width,
                       label='RFE', color=colors_rfe, alpha=0.85,
                       edgecolor='black', linewidth=1.2)
    
    axes[1].set_ylabel('R² per Feature (×1000)', fontweight='bold')
    axes[1].set_xlabel('Target', fontweight='bold')
    axes[1].set_title('Feature Efficiency: R² per Feature', fontweight='bold', fontsize=15)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels([t.capitalize() for t in efficiency_df['target']])
    axes[1].legend(frameon=True, fancybox=False, edgecolor='black', framealpha=0.95)
    axes[1].grid(True, alpha=0.25, linestyle='--', axis='y')
    
    # Add value labels
    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        axes[1].text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 0.05,
                    f"{bar1.get_height():.2f}", ha='center', va='bottom',
                    fontsize=10, fontweight='bold')
        axes[1].text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 0.05,
                    f"{bar2.get_height():.2f}", ha='center', va='bottom',
                    fontsize=10, fontweight='bold')
    
    plt.suptitle('Feature Efficiency Analysis', fontsize=18, fontweight='bold')
    plt.tight_layout()
    fig.savefig(FIGURES_DIR / 'test_feature_efficiency.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Saved: test_feature_efficiency.png")

## 7. Model Family Performance Comparison

In [ ]:
# Compare default vs tuned models
print("="*80)
print("DEFAULT VS TUNED MODELS COMPARISON")
print("="*80)

# Extract base model name and tuning status
df_test['model_base'] = df_test['model'].str.replace('_tuned', '')
df_test['is_tuned'] = df_test['model'].str.contains('_tuned')

tuning_comparison = df_test.groupby(['model_base', 'is_tuned', 'model_source'])['r2'].mean().reset_index()
tuning_pivot = tuning_comparison.pivot_table(
    index='model_base', 
    columns=['is_tuned', 'model_source'], 
    values='r2'
).round(4)

print("\nAverage R² across all targets:")
print(tuning_pivot)

# Calculate improvement from tuning
print("\n" + "="*80)
print("HYPERPARAMETER TUNING IMPACT")
print("="*80)

for model_base in tuning_pivot.index:
    print(f"\n{model_base}:")
    try:
        if (False, 'enhanced') in tuning_pivot.columns and (True, 'enhanced') in tuning_pivot.columns:
            default_enhanced = tuning_pivot.loc[model_base, (False, 'enhanced')]
            tuned_enhanced = tuning_pivot.loc[model_base, (True, 'enhanced')]
            if pd.notna(default_enhanced) and pd.notna(tuned_enhanced):
                improvement = (tuned_enhanced - default_enhanced) / default_enhanced * 100
                print(f"  Enhanced: {default_enhanced:.4f} → {tuned_enhanced:.4f} ({improvement:+.2f}%)")
        
        if (False, 'rfe') in tuning_pivot.columns and (True, 'rfe') in tuning_pivot.columns:
            default_rfe = tuning_pivot.loc[model_base, (False, 'rfe')]
            tuned_rfe = tuning_pivot.loc[model_base, (True, 'rfe')]
            if pd.notna(default_rfe) and pd.notna(tuned_rfe):
                improvement = (tuned_rfe - default_rfe) / default_rfe * 100
                print(f"  RFE:      {default_rfe:.4f} → {tuned_rfe:.4f} ({improvement:+.2f}%)")
    except:
        pass

In [ ]:
# Visualize model family performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Enhanced models
enhanced_avg = df_test[df_test['model_source'] == 'enhanced'].groupby('model')['r2'].mean().sort_values(ascending=False)
colors_bar = ['#27ae60' if '_tuned' in m else '#3498db' for m in enhanced_avg.index]

bars = axes[0].barh(range(len(enhanced_avg)), enhanced_avg.values, color=colors_bar,
                    alpha=0.85, edgecolor='black', linewidth=1.2)
axes[0].set_yticks(range(len(enhanced_avg)))
axes[0].set_yticklabels(enhanced_avg.index, fontsize=10)
axes[0].set_xlabel('Average R² (across all targets)', fontweight='bold')
axes[0].set_title('Enhanced Models (414 features)', fontweight='bold', fontsize=15)
axes[0].grid(True, alpha=0.25, linestyle='--', axis='x')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, enhanced_avg.values)):
    axes[0].text(val + 0.005, i, f'{val:.4f}', va='center', fontsize=9, fontweight='bold')

# RFE models
rfe_avg = df_test[df_test['model_source'] == 'rfe'].groupby('model')['r2'].mean().sort_values(ascending=False)
colors_bar = ['#27ae60' if '_tuned' in m else '#e74c3c' for m in rfe_avg.index]

bars = axes[1].barh(range(len(rfe_avg)), rfe_avg.values, color=colors_bar,
                    alpha=0.85, edgecolor='black', linewidth=1.2)
axes[1].set_yticks(range(len(rfe_avg)))
axes[1].set_yticklabels(rfe_avg.index, fontsize=10)
axes[1].set_xlabel('Average R² (across all targets)', fontweight='bold')
axes[1].set_title('RFE Models (reduced features)', fontweight='bold', fontsize=15)
axes[1].grid(True, alpha=0.25, linestyle='--', axis='x')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, rfe_avg.values)):
    axes[1].text(val + 0.005, i, f'{val:.4f}', va='center', fontsize=9, fontweight='bold')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#3498db', edgecolor='black', label='Default'),
    Patch(facecolor='#27ae60', edgecolor='black', label='Tuned')
]
axes[0].legend(handles=legend_elements, loc='lower right', frameon=True, 
              fancybox=False, edgecolor='black', framealpha=0.95)
axes[1].legend(handles=legend_elements, loc='lower right', frameon=True,
              fancybox=False, edgecolor='black', framealpha=0.95)

plt.suptitle('Model Performance Ranking (Average R² across targets)',
             fontsize=18, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'test_model_ranking.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved: test_model_ranking.png")

## 8. Comprehensive Summary Table

In [ ]:
# Create comprehensive summary for thesis table
summary_rows = []

for target in targets:
    target_df = df_test[df_test['target'] == target]
    
    for model_source in ['enhanced', 'rfe']:
        source_df = target_df[target_df['model_source'] == model_source]
        
        if len(source_df) > 0:
            # Get best model
            best = source_df.loc[source_df['r2'].idxmax()]
            
            # Get statistics
            summary_rows.append({
                'Target': target.capitalize(),
                'Model Source': 'Enhanced' if model_source == 'enhanced' else 'RFE',
                'Num Features': int(best['num_features']),
                'Best Model': best['model'],
                'R²': best['r2'],
                'RMSE': best['rmse'],
                'MAE': best['mae'],
                'Explained Var': best['explained_variance'],
                'R²/Feature (×1000)': (best['r2'] / best['num_features']) * 1000
            })

summary_table = pd.DataFrame(summary_rows)
summary_table = summary_table.round(4)

print("="*100)
print("COMPREHENSIVE SUMMARY - BEST MODELS PER TARGET AND SOURCE")
print("="*100)
print(summary_table.to_string(index=False))
print("="*100)

# Save to CSV
summary_path = RESULTS_DIR / 'test_evaluation_comprehensive_summary.csv'
summary_table.to_csv(summary_path, index=False)
print(f"\nSaved: {summary_path.name}")

## 9. Key Findings for Thesis

In [ ]:
print("""
═══════════════════════════════════════════════════════════════════════════════
                       KEY FINDINGS - TEST SET EVALUATION
═══════════════════════════════════════════════════════════════════════════════

1. OVERALL PERFORMANCE:
   • Test set: 86,453 held-out songs (never seen during training/validation)
   • Total evaluations: {} (Enhanced + RFE)
   • Best performing target: {} (R² = {:.4f})
   • Most challenging target: {} (R² = {:.4f})

2. ENHANCED MODELS (414 features):
   • Features: 23 audio+artist + 5 text + 2 sentiment + 384 embeddings
   • Average R² across all targets: {:.4f}
   • Best overall model: {}
   • Captures maximum information but computationally expensive

3. RFE MODELS (reduced features):
   • Feature reduction: 34-394 features per target ({}% avg reduction)
   • Average R² across all targets: {:.4f}
   • Performance trade-off: {:.2f}% avg R² loss for {}% fewer features
   • Significant efficiency gain: {}x R² per feature improvement

4. MODEL FAMILY INSIGHTS:
   • Gradient boosting (CatBoost, LightGBM, XGBoost) dominate performance
   • Hyperparameter tuning provides consistent improvements
   • Neural networks (MLPRegressor) competitive but require more tuning
   • Tree ensembles (ExtraTrees, RandomForest) solid baselines

5. FEATURE SELECTION IMPACT:
   • RFE successfully identifies most informative features
   • Energy prediction: 92% feature reduction, minimal R² loss
   • Danceability: 82% feature reduction, maintains strong performance
   • Popularity: Requires most features (minimal reduction effective)

6. PRACTICAL IMPLICATIONS:
   • RFE models recommended for production (faster inference, similar accuracy)
   • Enhanced models useful when maximum accuracy needed
   • Artist features contribute meaningfully (Experiment 2 improvement)
   • Target-specific feature selection crucial for efficiency

═══════════════════════════════════════════════════════════════════════════════
""".format(
    len(df_test),
    df_test.loc[df_test['r2'].idxmax(), 'target'].capitalize(),
    df_test['r2'].max(),
    df_test.loc[df_test['r2'].idxmin(), 'target'].capitalize(),
    df_test['r2'].min(),
    df_test[df_test['model_source'] == 'enhanced']['r2'].mean(),
    df_test[df_test['model_source'] == 'enhanced'].loc[df_test[df_test['model_source'] == 'enhanced']['r2'].idxmax(), 'model'],
    (1 - df_test[df_test['model_source'] == 'rfe']['num_features'].mean() / 414) * 100,
    df_test[df_test['model_source'] == 'rfe']['r2'].mean(),
    (df_test[df_test['model_source'] == 'enhanced']['r2'].mean() - df_test[df_test['model_source'] == 'rfe']['r2'].mean()) / df_test[df_test['model_source'] == 'enhanced']['r2'].mean() * 100,
    (1 - df_test[df_test['model_source'] == 'rfe']['num_features'].mean() / 414) * 100,
    (df_test[df_test['model_source'] == 'rfe']['r2'].mean() / df_test[df_test['model_source'] == 'rfe']['num_features'].mean()) / (df_test[df_test['model_source'] == 'enhanced']['r2'].mean() / 414)
))

print("\nAll figures saved to:", FIGURES_DIR)
print("\nFigures created:")
for fig_file in sorted(FIGURES_DIR.glob('test_*.png')):
    print(f"   {fig_file.name}")

print("\n" + "="*80)
print("TEST EVALUATION ANALYSIS COMPLETE - Ready for thesis integration")
print("="*80)